<a href="https://colab.research.google.com/github/Khalil-BenRomdhane/prediction_emboteillage/blob/main/prediction_emboteillage_off.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

importation des modules necessaires

In [ ]:
#%pip install pandas numpy  IPython matplotlib scikit-learn  seaborn


In [ ]:
#%pip install missingno

In [ ]:
#%pip install geopy

Importation des modules

In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import GoogleV3
import matplotlib.pyplot as plt
import gc
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import missingno as msno

Importation de la dataset

In [ ]:
file_path = 'C:/Users/asus/Desktop/jam-prediction/comptages-routiers-permanents.csv'
df=pd.read_csv(file_path,sep=";")

Affichage des 3 premieres lignes

In [ ]:
df.head()

Les dimennsions de la dataset

In [ ]:
df.shape

Extraction des rues existants dans la datset

In [ ]:
rues_paris = df["Libelle"].unique()

Les dimensions de la serie "rues_paris" (Nbr de rues)

In [ ]:
rues_paris.shape

Affichage des rues

In [ ]:
rues_paris

Affichage du nombre des valeurs nul pour chaque colonne

In [ ]:
df.isnull().sum()

In [ ]:
#plt.figure(figsize=(10, 6))
#sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
#plt.title('Missing Values Heatmap')
#plt.show()

----PARTIE MANIPULATION DES DONNEES----

Creation des colonnes Longitude et Latitude a travers la colonne geo_point_2D

In [ ]:
df[['Latitude', 'Longitude']] = df['geo_point_2d'].str.split(', ', expand=True)
df.head()

Supression de la colonne geo_point_2d

In [ ]:
df.drop(columns=['geo_point_2d'], inplace=True)

Affichage des 5 premiers lignes de la dataset pour voir les nouveaux changement

In [ ]:
df.head()

Extraction des valeurs unique de la colonne "Etat trafic"

In [ ]:
Etat_trafic=df["Etat trafic"].unique()
Etat_trafic_mapping={}

Attribution d'une valeur numerique pour chaque valeur unique de la colonne "Etat trafic" et les enregistrer dans un dictionnaire

In [ ]:
for etat,i in zip(Etat_trafic,range(1,len(Etat_trafic)+1)):
    Etat_trafic_mapping[etat]=i
Etat_trafic_mapping

Creation d'une nouvelle colonne pour les valeurs numerique de la colonne "Etat trafic"

In [ ]:
df['Encoded_Traffic'] = df['Etat trafic'].map(Etat_trafic_mapping)
df.head()

In [ ]:
df.rename(columns={"Encoded_Traffic": "Etat_trafic_encoded"}, inplace=True)
df.drop(columns=['Etat trafic'], inplace=True)

In [ ]:
Libelle=df["Libelle"].unique()
Libelle_mapping={}

In [ ]:
for libelle,i in zip(Libelle,range(1,len(Libelle)+1)):
    Libelle_mapping[libelle]=i
Libelle_mapping

In [ ]:
df['Libelle_encoded'] = df['Libelle'].map(Libelle_mapping)

In [ ]:
df.head()

In [ ]:
df.drop(columns=['Libelle'], inplace=True)

In [ ]:
df.head()

In [ ]:
Libelle_noeud_amont=df["Libelle noeud amont"].unique()
Libelle_noeud_amont_mapping={}

In [ ]:
for amont,i in zip(Libelle_noeud_amont,range(1,len(Libelle_noeud_amont)+1)):
    Libelle_noeud_amont_mapping[amont]=i
Libelle_noeud_amont_mapping

In [ ]:
df['Libelle_noeud_amont_encoded'] = df['Libelle noeud amont'].map(Libelle_noeud_amont_mapping)

In [ ]:
df.drop(columns=['Libelle noeud amont'], inplace=True)

In [ ]:
df.head()

In [ ]:
df.rename(columns={"Etat_traffic": "Etat_trafic_encoded"}, inplace=True)

In [ ]:
df.head(2)

In [ ]:
Libelle_noeud_aval=df["Libelle noeud aval"].unique()
Libelle_noeud_aval_mapping={}

In [ ]:
for aval,i in zip(Libelle_noeud_aval,range(1,len(Libelle_noeud_aval)+1)):
    Libelle_noeud_aval_mapping[aval]=i
Libelle_noeud_aval_mapping

In [ ]:
df['Libelle_noeud_aval_encoded'] = df['Libelle noeud aval'].map(Libelle_noeud_aval_mapping)

In [ ]:
df.drop(columns={"Libelle noeud aval"},inplace=True)

In [ ]:
df.head()

In [ ]:
df['Date et heure de comptage'] = pd.to_datetime(df['Date et heure de comptage'],utc=True)

In [ ]:
df.head()


In [ ]:
df['Year'] = df['Date et heure de comptage'].dt.year
df['Month'] = df['Date et heure de comptage'].dt.month
df['Day'] = df['Date et heure de comptage'].dt.day
df['Hour'] = df['Date et heure de comptage'].dt.hour
df['jour de la semaine']=df['Date et heure de comptage'].dt.day_of_week

In [ ]:
df.head()

In [ ]:
df.set_index("Date et heure de comptage",inplace=True)

In [ ]:
print(f"Data Available from {df.index.min()} to {df.index.max()}")

In [ ]:
idx = pd.date_range(df.index.min(), df.index.max())
idx

In [ ]:
# Calculate percentage of missing values in each column
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Create a bar plot of missing values
plt.figure(figsize=(10, 6))
sns.barplot(x=missing_percentage.index, y=missing_percentage)
plt.xticks(rotation=90)
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing Values')
plt.title('Percentage of Missing Values in Each Column')
plt.show()


In [ ]:
 #Select the features for KNN imputation
features = [ 'Débit horaire', 'Taux d\'occupation', 'Latitude', 'Longitude','Year','Month','Hour','Day','jour de la semaine']

# Standardize the features
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df[features])

# Initialize the KNN imputer
imputer = KNNImputer(n_neighbors=5)

# Apply the imputer to the standardized data
data_imputed = imputer.fit_transform(data_scaled)

# Convert the imputed data back to a DataFrame
data_imputed_df = pd.DataFrame(data_imputed, columns=features)

#Reverse the standardization
data_imputed_df = pd.DataFrame(scaler.inverse_transform(data_imputed_df), columns=features)

# Replace the original columns with the imputed values
df[features] = data_imputed_df[features]

# Inspect the imputed data
print(df.head())

In [ ]:
#Returning a data set with oaccupation rate per month for each road
occupation_mois= df.groupby(["Day","Month",'Year','Libelle_encoded'])['Taux d\'occupation'].mean().reset_index()
occupation_mois.set_index(['Day','Month', 'Year'], inplace=True)
occupation_moyenne_par_mois_libelle = occupation_mois[occupation_mois['Libelle_encoded']==2]
occupation_moyenne_par_mois_libelle.sort_index(level=['Day','Month', 'Year'],ascending=[True,True,True],inplace=True)
occupation_moyenne_par_mois_libelle.head(20)


In [ ]:

# Générer les étiquettes pour l'axe des x au format "mois-année"
xticks = occupation_moyenne_par_mois_libelle.index.map(lambda idx: f"{idx[1]}")

# Créer une séquence d'entiers pour l'axe des x
x_values = range(len(occupation_moyenne_par_mois_libelle))

# Créer le graphique
plt.figure(figsize=(12, 8))
plt.plot(x_values, occupation_moyenne_par_mois_libelle['Taux d\'occupation'], marker='o')

# Étiqueter les axes des x avec les mois et années
plt.xticks(ticks=x_values, labels=xticks, rotation=90)

# Ajouter un titre et étiquettes des axes
plt.title('Taux d\'occupation moyen par mois et par année pour un libelle specifique')
plt.xlabel('Mois-Année')
plt.ylabel('Taux d\'occupation moyen (%)')

# Ajouter une grille
plt.grid(True)

# Ajuster la disposition pour éviter le chevauchement des étiquettes
plt.tight_layout()

# Afficher le graphique
plt.show()



In [ ]:
 #df.set_index("Date et heure de comptage", inplace=True)

In [ ]:
#df.head()

In [ ]:
#df.sort_index(inplace=True)
#df.head()

In [ ]:
#df_0=df.fillna({'Taux d\'occupation': 0, 'Débit horaire': 0})

In [ ]:
#gr_0=df_0.groupby(["Libelle_encoded","Date et heure de comptage"]).agg({'Débit horaire':'mean',
                                      #'Taux d\'occupation':'mean'}).reset_index()

In [ ]:
 #taux_occupation_na_0_count=gr_0[gr_0["Taux d'occupation"]==0].shape[0]
 #debit_horaire_na_0_count=gr_0[gr_0["Débit horaire"]==0].shape[0]

 #print(debit_horaire_na_0_count,taux_occupation_na_0_count)


In [ ]:
 #gr_0["Taux d'occupation"].replace(0, np.nan, inplace=True)
 #gr_0["Débit horaire"].replace(0, np.nan, inplace=True)

In [ ]:
 #gr=df.groupby(["Libelle_encoded","Date et heure de comptage"]).agg({'Débit horaire':'mean',
                                       #'Taux d\'occupation':'mean'}).reset_index()

In [ ]:
 #taux_occupation_na_count=df["Taux d'occupation"].isna().sum()
 #debit_horaire_na_count=df["Débit horaire"].isna().sum()

 #print(debit_horaire_na_count,taux_occupation_na_count)



In [ ]:
 #gr_0

In [ ]:
  # def concat_coordinates(coords):
    # Remplacer les valeurs nulles ou NaN par des chaînes vides
     #coords = coords.fillna('')
    # Concaténer les coordonnées en une seule chaîne séparée par '|'
     #return '|'.join(coords)

In [ ]:
 #longitude_latitude= df[['Libelle_encoded', 'Longitude', 'Latitude']]
 #longitude_latitude['Coordinates'] = df['Longitude'] +","+ df['Latitude']
 #longitude_latitude.drop(columns=["Longitude","Latitude"],inplace=True)

In [ ]:
 #grouped_longitude_latitude= longitude_latitude.groupby(['Libelle_encoded'])['Coordinates'].agg(concat_coordinates).reset_index()

In [ ]:
 #grouped_longitude_latitude

In [ ]:
 #final_data_set = pd.merge(gr_0, grouped_longitude_latitude, left_on="Libelle_encoded", right_on='Libelle_encoded', how='inner')

In [ ]:
 #final_data_set

In [ ]:
 #df_str=df
 #df_str["Identifiant arc"]=df_str['Identifiant arc'].astype(str)
 #grouped_id_arc= df_str.groupby(['Libelle_encoded'])['Identifiant arc'].agg(concat_coordinates).reset_index()

In [ ]:
 #grouped_id_arc

In [ ]:
 #final_data_set = pd.merge(final_data_set, grouped_id_arc, left_on="Libelle_encoded", right_on='Libelle_encoded', how='inner')

In [ ]:
 #final_data_set

In [ ]:

 #df_str["Identifiant noeud amont"]=df_str['Identifiant noeud amont'].astype(str)
 #grouped_id_amont_arc= df_str.groupby(['Libelle_encoded'])['Identifiant noeud amont'].agg(concat_coordinates).reset_index()

In [ ]:
 #grouped_id_amont_arc= df_str.groupby(['Libelle_encoded'])['Identifiant noeud amont'].agg(concat_coordinates).reset_index()

In [ ]:
 #grouped_id_amont_arc

In [ ]:
 #final_data_set = pd.merge(final_data_set, grouped_id_amont_arc, left_on="Libelle_encoded", right_on='Libelle_encoded', how='inner')

In [ ]:
 #final_data_set

In [ ]:
 #df_str["Identifiant noeud aval"]=df_str['Identifiant noeud aval'].astype(str)
 #grouped_id_aval_arc= df_str.groupby(['Libelle_encoded'])['Identifiant noeud aval'].agg(concat_coordinates).reset_index()

In [ ]:
 #grouped_id_aval_arc


In [ ]:
 #final_data_set = pd.merge(final_data_set, grouped_id_aval_arc, left_on="Libelle_encoded", right_on='Libelle_encoded', how='inner')

In [ ]:
 #final_data_set

In [ ]:
 #df_str["Libelle_noeud_amont_encoded"]=df_str['Libelle_noeud_amont_encoded'].astype(str)
 #grouped_id_aval_arc= df_str.groupby(['Libelle_encoded'])['Libelle_noeud_amont_encoded'].agg(concat_coordinates).reset_index()

In [ ]:
 #final_data_set = pd.merge(final_data_set, grouped_id_aval_arc, left_on="Libelle_encoded", right_on='Libelle_encoded', how='inner')

In [ ]:
 #final_data_set

In [ ]:
 #df_str["Libelle_noeud_aval_encoded"]=df_str['Libelle_noeud_aval_encoded'].astype(str)
 #grouped_id_aval_arc= df_str.groupby(['Libelle_encoded'])['Libelle_noeud_aval_encoded'].agg(concat_coordinates).reset_index()

In [ ]:
 #final_data_set = pd.merge(final_data_set, grouped_id_aval_arc, left_on="Libelle_encoded", right_on='Libelle_encoded', how='inner')

In [ ]:
 #final_data_set

In [ ]:
 #final_data_set.set_index("Date et heure de comptage",inplace=True)

In [ ]:
 #final_data_set

In [ ]:
 #def parse_coordinates(coord_string):
     #pairs = coord_string.split('|')
     #coordinates = [(float(pair.split(',')[1]), float(pair.split(',')[0])) for pair in pairs]
     #return coordinates


In [ ]:
 #final_data_set['Coordinates'] = final_data_set['Coordinates'].apply(parse_coordinates)
 #final_data_set['Latitudes'] = final_data_set['Coordinates'].apply(lambda x: [coord[0] for coord in x])
 #final_data_set['Longitudes'] = final_data_set['Coordinates'].apply(lambda x: [coord[1] for coord in x])
 #final_data_set['Longitudes'] = final_data_set['Coordinates'].apply(lambda x: [coord[1] for coord in x])

In [ ]:
 #df_KNN=df.drop(columns=["Date debut dispo data","Date fin dispo data",
   #                      "geo_shape","Etat arc","Identifiant noeud amont",
     #                    "Identifiant noeud aval","Libelle_noeud_amont_encoded",
       #                  "Libelle_noeud_aval_encoded","Etat_trafic_encoded",
         #                "Etat_trafic_encoded","Libelle_encoded","Identifiant arc"])
 #df_KNN.reset_index(inplace=True)

In [ ]:
 #df_KNN["Date et heure de comptage"]=pd.to_datetime(df_KNN['Date et heure de comptage'])

In [ ]:
 #df_KNN['Heure de comptage'] = df_KNN['Date et heure de comptage'].dt.hour
 #df_KNN['Jour de comptage'] = df_KNN['Date et heure de comptage'].dt.day
 #df_KNN['Mois de comptage'] = df_KNN['Date et heure de comptage'].dt.month
 #df_KNN['Année de comptage']=df_KNN['Date et heure de comptage'].dt.year
 #df_KNN['Jour de la semaine'] = df_KNN['Date et heure de comptage'].dt.weekday
 #df_KNN.drop(columns=["Date et heure de comptage"],inplace=True)


In [ ]:
 #df_KNN.head()

In [ ]:
 #imputer = KNNImputer(n_neighbors=10)
 #df_KNN_imputed = imputer.fit_transform(df_KNN)
 #df_KNN_imputed = pd.DataFrame(df_KNN_imputed, columns=df_KNN.columns)

In [ ]:
 #mean_occupation_rate = df.groupby(['Libelle_encoded', 'Date et heure de comptage'])["Taux d'occupation"].mean().reset_index()


In [ ]:
 #mean_occupation_rate

In [ ]:
 #def traffic_evaluation(x):
   #  if  x>=0 and x<15 :
     #    return 1
     #elif x>=15 and x<30 :
       #  return 2
    #elif x>=30 and x<50 :
     #   return 3
    #elif x>=50 :
     #   return 4

In [ ]:
#mean_occupation_rate.set_index("Date et heure de comptage",inplace=True)

In [ ]:
#mean_occupation_rate.info()

In [ ]:
#mean_occupation_rate.head()


In [ ]:
#mean_occupation_rate = mean_occupation_rate.assign(Etat_trafic_encoded=mean_occupation_rate["Taux d'occupation"].apply(traffic_evaluation))


In [ ]:
#mean_occupation_rate.head()


In [ ]:
#mean_occupation_rate["Etat_trafic_encoded"]

In [ ]:
#longitude_latitude

In [ ]:
#grouped_longitude_latitude

In [ ]:
#final_data_set = pd.merge(mean_occupation_rate, grouped_longitude_latitude, left_on="Libelle_encoded", right_on='Libelle_encoded', how='inner')

In [ ]:
#final_data_set.set_index(mean_occupation_rate.index,inplace=True)

In [ ]:
#final_data_set

In [ ]:
#final_data_set["Libelle_encoded"].unique()

In [ ]:
#mean_debit_horaire = df.groupby(['Libelle_encoded', 'Date et heure de comptage'])["Débit horaire"].mean().reset_index()


In [ ]:
#del df
#del mean_occupation_rate
#del grouped_longitude_latitude
#del longitude_latitude
#gc.collect()

In [ ]:
#mean_debit_horaire.set_index("Date et heure de comptage",inplace=True)

In [ ]:
#mean_debit_horaire

In [ ]:
#final_data_set

In [ ]:
jours_feries=pd.read_csv("C:/Users/asus/Desktop/Jam-prediction/jours_feries_metropole.csv",sep=",")

In [ ]:
jours_feries

In [ ]:
jours_feries['date'] = pd.to_datetime(jours_feries['date'])

In [ ]:
jours_feries[jours_feries["date"]=="2024-01-01"]

In [ ]:
jours_feries['date']

In [ ]:
def is_holiday(date):
        date = date.strftime('%Y-%m-%d')
        return date in jours_feries["date"].astype(str).values

In [ ]:
df.head()

In [ ]:
df['feries'] =df.index.map(is_holiday)